In [1]:
from preprocessing.prepro_utils import remove_missing_description, conv_scode_desc, remove_neg_quan, get_sub_dataset, get_transactions, gen_transactions_csv, map_stock_codes, map_to_json
import pandas as pd
from ydata_profiling import ProfileReport


In [2]:
original = pd.read_excel('./original.xlsx')

In [3]:
ProfileReport(original)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

<div style="background-color: rgba(150, 0,50, .5); border-radius: 20px; padding: 20px; color: white" >

<ul>
    <li>
        this report shows that three columns are of unsupported type: 'StockCode', 'InvoiceNo' and 'Description', which is a result of the fact that these columns contain data of different datatypes
    </li>
    </br>
    <li>
        the 'InvoiceNo' is not needed, so it will be deleted, the other two columns will be converted into String datatype
    </li></br>
    <li>
        before converting datatypes, a look into missing values tells us that the Description column has 0.3% missing data, if the description of the product is missing, the row will not provide any value for the study, therefore we'll delete it
    </li></br>
    <li>
        another problem we see is that the CustomerId field has a lot of missing values, to address this we'll split the logic into two loops, one to extract items from the subset with known CustomerID, and another to extract from the subset with missing CustomerID
    </li></br>
    <li>
        rows with missing CustomerId should not be deleted because it only signifies that a client bought something without being registered to the store system, the list of bought items is still important
    </li></br>
    <li>
        negative quantity values signifies returned items, which are not a subject for this study, therefore they'll be removed
    </li>
</ul>






</div>



# removing rows with missing Description

In [4]:
remove_missing_description(original)

# converting SotckCode and Description into String

In [5]:
conv_scode_desc(original)

# Removing negative quantities

In [6]:
remove_neg_quan(original)

In [7]:
ProfileReport(original)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
subset = get_sub_dataset(original, ['InvoiceNo', 'Quantity', 'UnitPrice', 'Country'])

# need to get what every Customer bought => unique by CustomerID
# but a customer can buy a lot of items at the time, different rows with same CustomerID represent one trasaction
# unique by time such that all transactions made in the range of X minutes will be considered as one transaction

# extract list of transactions made by anonymous customers
## in this case we will take each individual code as a transaction

In [9]:
#transactions.append([customer_['StockCode']] for _, customer_ in original_[original_["CustomerID"].isna()].iterrows())

# extract the transactions list made by registered customers

In [10]:
# the time interval for transaction grouping in minutes
# 3

In [11]:
transactions = get_transactions(subset, 10)

In [12]:
len(transactions)

26531

In [13]:
gen_transactions_csv(transactions,'.')

writen to ./transactions.csv


<div style="background-color: rgba(0, 255,50, .5); border-radius: 20px; padding: 20px; color: white" >
    because we used the StockCode of items, we will need to map each StockCode to its corresponding item 
</div>



# we'll write the map to a json file in case we need it later for the user interface (search products)

In [14]:
mapping = map_stock_codes(subset)
map_to_json(mapping, '.')
print(mapping)

[['10002', 'INFLATABLE POLITICAL GLOBE'], ['10080', 'GROOVY CACTUS INFLATABLE'], ['10120', 'DOGGY RUBBER'], ['10123C', 'HEARTS WRAPPING TAPE'], ['10124A', 'SPOTS ON RED BOOKCOVER TAPE'], ['10124G', 'ARMY CAMO BOOKCOVER TAPE'], ['10125', 'MINI FUNKY DESIGN TAPES'], ['10133', 'COLOURING PENCILS BROWN TUBE'], ['10135', 'COLOURING PENCILS BROWN TUBE'], ['11001', 'ASSTD DESIGN RACING CAR PEN'], ['15030', 'FAN BLACK FRAME'], ['15034', 'PAPER POCKET TRAVELING FAN'], ['15036', 'ASSORTED COLOURS SILK FAN'], ['15039', 'SANDALWOOD FAN'], ['15044A', 'PINK PAPER PARASOL'], ['15044B', 'BLUE PAPER PARASOL'], ['15044C', 'PURPLE PAPER PARASOL'], ['15044D', 'RED PAPER PARASOL'], ['15056BL', 'EDWARDIAN PARASOL BLACK'], ['15056N', 'EDWARDIAN PARASOL NATURAL'], ['15056P', 'EDWARDIAN PARASOL PINK'], ['15056bl', 'EDWARDIAN PARASOL BLACK'], ['15056n', 'EDWARDIAN PARASOL NATURAL'], ['15056p', 'EDWARDIAN PARASOL PINK'], ['15058A', 'BLUE POLKADOT GARDEN PARASOL'], ['15058B', 'PINK POLKADOT GARDEN PARASOL'], ['15

In [15]:
import sqlite3
conn = sqlite3.connect(':memory:')
original.to_sql('my_table', conn, index=False)

540455

In [16]:


# Execute SQL queries on the DataFrame
query1 = 'SELECT StockCode, sum(Quantity) FROM my_table  group by (StockCode)'
result1 = pd.read_sql_query(query1, conn)
print(f"Result of Query 1:\n{result1}\n")


# Close the database connection
conn.close()

Result of Query 1:
         StockCode  sum(Quantity)
0            10002          860.0
1            10080          325.0
2            10120          193.0
3           10123C            5.0
4           10124A           16.0
...            ...            ...
3953  gift_0001_20           20.0
3954  gift_0001_30            7.0
3955  gift_0001_40            3.0
3956  gift_0001_50            4.0
3957             m            1.0

[3958 rows x 2 columns]



In [17]:
sum = 0
for index, row in original.iterrows():
    if row["StockCode"] == "10123C" : 
        print(f"found at {index} with quality: {row['Quantity']}")
        sum += row["Quantity"]
    
print(sum)

    

found at 5451 with quality: 1.0


found at 109611 with quality: 3.0
found at 140929 with quality: 1.0
5.0
